In [ ]:
import numpy as np
import pandas as pd
X_trainpd = pd.read_csv("UCI HAR Dataset/train/X_train.txt",delim_whitespace=True, header=None).values
y_trainpd = pd.read_csv("UCI HAR Dataset/train/y_train.txt",delim_whitespace=True, header=None).values.ravel()
X_testpd = pd.read_csv("UCI HAR Dataset/test/X_test.txt",delim_whitespace=True, header=None).values
y_testpd = pd.read_csv("UCI HAR Dataset/test/y_test.txt",delim_whitespace=True, header=None).values.ravel()
X_combined = pd.concat([pd.DataFrame(X_trainpd), pd.DataFrame(X_testpd)])
y_combined = pd.concat([pd.Series(y_trainpd), pd.Series(y_testpd)])

In [ ]:
from deap import base, creator, tools, algorithms
num_features = X_combined.shape[1]
def attribute_from_dataset():
    return X_combined

In [ ]:
def gaussian_membership(x, x_hat, theta):
    return np.exp(-((x - x_hat) ** 2) / (2 * theta ** 2))

In [ ]:
def fuzzy_rules(features):
    average, maximum, minimum, std_deviation, variance = features
    fuzzy_classification = None
    if all(avg > -0.6 and mini > 0.2 and maxi > -0.7 and std > 0.1 for avg, mini, maxi, var,std in zip(average, minimum, maximum, variance, std_deviation)):
        fuzzy_classification = "BEST"
    elif all(avg > -0.6 and mini > 0.2 and maxi > -0.7 and std <=0.1 for avg, mini, maxi,var, std in zip(average, minimum, maximum, variance, std_deviation)):
        fuzzy_classification = "GOOD"
    else:
        fuzzy_classification = "BAD"
    return fuzzy_classification

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_float",attribute_from_dataset)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float,n=num_features)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
def fuzzy_fitness(individual):
    selected_columns = [i for i in range(561)]
    feature=np.take(X_combined,selected_columns,axis=1)
    average = np.mean(feature,axis=0)
    maximum = np.max(feature,axis=0)
    minimum = np.min(feature,axis=0)
    std_deviation = np.std(feature,axis=0)
    variance = np.var(feature,axis=0)
    features=[average,maximum,minimum,std_deviation,variance]
    fuzzy_classification = fuzzy_rules(features)
    gaussian_values = []
    for col_avg, col_std in zip(average, std_deviation):
        gaussian_value = gaussian_membership(col_avg, 0.5, col_std)
        gaussian_values.append(gaussian_value)
    gaussian_value=np.prod(gaussian_values)
    if fuzzy_classification == 'BEST':
        fitness_value = 0.4 *gaussian_value
    elif fuzzy_classification == 'GOOD':
        fitness_value = 0.3 *gaussian_value
    else:
        fitness_value = 0.3
    return (fitness_value,)

In [ ]:
toolbox.register("evaluate", fuzzy_fitness)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.2)

In [ ]:
def tournament_selection(population, k):
    tournsize = 5 # Specify the value of tournsize here
    selected = []
    indices = list(range(len(population)))
    for _ in range(k):
        competitors_indices = np.random.choice(indices, tournsize, replace=False)
        competitors = [population[i] for i in competitors_indices]
        selected.append(max(competitors, key=lambda ind: ind.fitness.values[0]))
    return selected

In [ ]:
toolbox.register("select", tournament_selection)
hof = tools.HallOfFame(1)

In [ ]:
population_size = X_combined.shape[0] # Set population size equal to the number of rows in your dataset
population = toolbox.population(n=population_size) # Initialize population with rows of the dataset
population,log=algorithms.eaMuPlusLambda(population, toolbox, mu=population_size,lambda_=2*population_size, cxpb=0.7, mutpb=0.2, ngen=0, stats=None, halloffame=hof)
best_individuals = hof
df1=pd.DataFrame(population[0])
df2=pd.DataFrame(population[0][0])
df2.to_csv('geneticalgooutput.csv')